In [1]:
import numpy as np
from keras.datasets import cifar10
from keras import models
from keras import layers
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# Image enlargement
import scipy.misc
big_x_train = np.array([scipy.misc.imresize(x_train[i], (64, 64, 3)) 
                            for i in range(len(x_train))]).astype('float32')

In [ ]:
img_rows, img_cols, channels = big_x_train.shape[1], big_x_train.shape[2], big_x_train.shape[3]
img_rows, img_cols, channels

In [ ]:
vgg_model = VGG19(include_top = False, input_shape = (img_rows, img_cols, channels))

In [ ]:
# Generate features from VGG16 model
X_train_vgg_features = vgg_model.predict(big_x_train)

In [ ]:
def extract_images_features(imgs, target_size, model):
    """Increase image size to target size and 
    extract image features from model"""
    large_imgs = np.array([scipy.misc.imresize(img, target_size) for img in imgs])
    features = model.predict(large_imgs)
    return features

In [ ]:
X_test_vgg_features = extract_images_features(x_test, (64,64,3), vgg_model)

In [ ]:
X_test_vgg_features.shape

### Transfer learning

In [ ]:
# Transfer learning
model = models.Sequential()
model.add(layers.Conv2D(filters=128, kernel_size=2, input_shape=X_train_vgg_features.shape[1:]))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='rmsprop',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])
# Training
model.fit(X_train_vgg_features, y_train, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test_vgg_features, y_test)
accuracy

### Inception model

In [2]:
img_rows, img_cols, channels = 139, 139, 3
# img_rows, img_cols, channels = 299, 299, 3

from keras.applications.inception_v3 import InceptionV3, preprocess_input
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channels))

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
# reshape images dimension to 139, 139, 3
import scipy.misc
big_x_train = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels)) 
                            for img in x_train]).astype('float32')
inception_input_train = preprocess_input(big_x_train)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  after removing the cwd from sys.path.


In [8]:
X_train_inception_features = inception_model.predict(inception_input_train)

In [16]:
from keras.preprocessing.image import ImageDataGenerator
# REF: https://keras.io/preprocessing/image/
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(inception_input_train)

In [17]:
def feature_generation(orig_x, orig_y, samples_count, batch_size=32):
    assert samples_count % batch_size == 0
    features = np.zeros(shape=(samples_count, 3, 3, 2048))
    labels = np.zeros(shape=(samples_count, 1))
    i = 0
    for input_batch, labels_batch in datagen.flow(x=orig_x, y=orig_y, batch_size=batch_size, seed=100):
        resized = np.array([scipy.misc.imresize(img, (139, 139, 3)) 
                            for img in input_batch]).astype('float32')
        
        features_batch = inception_model.predict(preprocess_input(resized))
        
        features[i:i+batch_size] = features_batch
        labels[i:i+batch_size] = labels_batch
        i += batch_size
        
        if i >= samples_count:
            break
    return features, labels
    

In [18]:
X_train_augmented, y_train_augmented = feature_generation(x_train, y_train, 10000, batch_size=50)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


In [19]:
# stack together
X_train = np.vstack([X_train_augmented, X_train_inception_features])
y_train = np.vstack([y_train_augmented, y_train, ])

In [21]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
# np.savez('inception_features_train', features=X_train_inception_features)

In [ ]:
# X_train_inception_features = extract_images_features(x_train, (img_rows, img_cols, channels), inception_model)
# X_test_inception_features = extract_images_features(x_test, (img_rows, img_cols, channels), inception_model)

#### Model building, loss function and optimizer

In [26]:
import keras.optimizers
model = models.Sequential()
model.add(layers.Conv2D(filters=128, kernel_size=2, input_shape=X_train.shape[1:]))
model.add(layers.Dropout(0.4))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(lr=1e-4),
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

model.summary()

TypeError: softmax() got an unexpected keyword argument 'axis'

#### Training

In [ ]:
# Training
model.fit(X_train_inception_features, y_train, epochs=20, batch_size=128, validation_split=0.2, verbose=2)

In [ ]:
# Training
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

#### Bottle neck feature for tests

In [ ]:
big_x_test = np.array([scipy.misc.imresize(img, (139, 139, 3)) 
                            for img in x_test]).astype('float32')
# Important! preprocess input data!
inception_input_test = preprocess_input(big_x_test)

In [ ]:
X_test_inception_features = inception_model.predict(inception_input_test)

In [ ]:
y_test = to_categorical(y_test)
loss, accuracy = model.evaluate(X_test_inception_features, y_test)
accuracy

## Data-agmentation with transfer learning

In [3]:
# build model with conv_base of Inception V3
img_rows, img_cols, channels = 96, 96, 3

from keras.applications.vgg16 import VGG16, preprocess_input
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, channels))

In [8]:
# build network
network = models.Sequential()
network.add(vgg_model)
network.add(layers.Flatten())
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(10, activation='sigmoid'))

In [9]:
vgg_model.trainable = False

In [10]:
# compile the network
import keras.optimizers
network.compile(optimizer=keras.optimizers.RMSprop(lr=2e-5),
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

In [11]:
len(network.trainable_weights)

4

In [ ]:
import scipy.misc
# Input Pixel Augmentation
x_train_vgg = np.array([scipy.misc.imresize(img, (img_rows, img_cols, channels)) 
                            for img in x_train]).astype('float32')
# Important! preprocess input data!
x_train_vgg = preprocess_input(x_train_vgg)

In [4]:
def preprocess_data(X, img_rows, img_cols, channels):
    from skimage.transform import resize
    # Input Pixel Augmentation
    X_res = np.array([resize(img, (img_rows, img_cols, channels)) 
                            for img in X]).astype('float32')
    # Important! preprocess input data!
    return preprocess_input(X_res)

In [ ]:
x_train_vgg.shape

In [5]:
# Do the same thing for X_train and X_test
x_train_vgg = preprocess_data(x_train, 96, 96, 3)
x_test_vgg = preprocess_data(x_test, 96, 96, 3)

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [6]:
x_test_vgg.shape

(10000, 96, 96, 3)

In [7]:
# one-hot encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
# data generator

from keras.preprocessing.image import ImageDataGenerator
# REF: https://keras.io/preprocessing/image/
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)


In [13]:
train_generator = datagen.flow(x_train_vgg, y_train, batch_size=32, subset='training')
valid_generator = datagen.flow(x_train_vgg, y_train, batch_size=32, subset='validation')

In [14]:
history = network.fit_generator(
        train_generator,
        steps_per_epoch = 200,
        epochs = 5,
        validation_data = valid_generator,
        validation_steps = 50
)

Epoch 1/5


/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:959: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image.py:967: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


 78/200 [==========>...................] - ETA: 5:33 - loss: 2.3223 - acc: 0.0938

KeyboardInterrupt: 